# Kepler Timeseries2Score musification. Kepler Objects of Interest (KOI) catalog. Samples:1-100

Data download: https://archive.stsci.edu/pub/kepler/lightcurves/
#### ACKNOWLEDGMENT
This research includes data collected with the Kepler mission, obtained from the MAST data archive at the Space Telescope Science Institute (STScI). Funding for the Kepler mission is provided by the NASA Explorer Program and by the NASA Science Mission Directorate. STScI is operated by the Association of Universities for Research in Astronomy, Inc., under NASA contract NAS 5–26555.


In [ ]:
import astropy
from astropy.timeseries import TimeSeries

from astropy import units as u
from astropy.timeseries import BoxLeastSquares
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import aggregate_downsample

import matplotlib.pylab as plt
from scipy.signal import find_peaks

import numpy as np

import os
from pathlib import Path

import random

import time
from pythonosc import udp_client

import math
from math import log2, pow

## Checking the library

In [ ]:
root = '## YOUR FOLDER PATH TO THE DOWNLOADED SPECTRA ##'

In [ ]:
for path, subdirs, files in os.walk(root):
    for name in files:
        print([os.path.join(path, name)])

In [ ]:
len(files)

In [ ]:
file = [os.path.join(path, name)]
str = " " 
Ffile = (str.join(file))  

route = Path(Ffile)

Fname = route.with_suffix('')

In [ ]:
NumSamp=100
sample = files [0:NumSamp]
sample

## Periodogram analysis for each light curve

In [ ]:
curves=0
freq = np.zeros((len(sample), 1))
dur = np.zeros((len(sample), 1))
amp = np.zeros((len(sample), 1))

i = 0
tempo = 1
for path, subdirs, files in os.walk(root):
    for name in sample: #changing "sample" by "files" explores the whole folder
        curves=curves+1
        file = [os.path.join(path, name)]
        str = " " 
        Ffile = (str.join(file))
        route = Path(name)
        Fname = route.with_suffix('')
        Fpng = route.with_suffix('.png')

#Graph
        filename = Ffile
        ts = TimeSeries.read(filename, format='kepler.fits')  
        
        fig, ax = plt.subplots(3, 1, figsize=(8, 12))
        fig.suptitle(Fname, size=14)
        fig.subplots_adjust(hspace=0.35, wspace=0.15, left=0.07, right=0.97)
        
        ax[0].plot(ts.time.jd, ts['sap_flux'], 'k.', markersize=1, color='blue')
        ax[0].set_xlabel('Julian Date')
        ax[0].set_ylabel('SAP Flux (e-/s)')
    
        
        model = BoxLeastSquares.from_timeseries(ts, 'sap_flux') 
        
        periodogram = model.autopower(0.2 * u.day)  
        max_samp = np.argmax(periodogram.power) 
        
        period = periodogram.period[max_samp]

        dyn = periodogram.power[max_samp]
        
        duration = periodogram.transit_time[max_samp]  

#To obtain a graphical representation of the light curve, its periodogram and folded curve, uncomment "plt.savefig('Spectra.png')"
          
        ax[1].plot(periodogram.period, periodogram.power, 'k.', markersize=1)
        ax[1].set_xlabel('Period (d)')
        ax[1].set_ylabel('Power')
        ax[1].axvline(period.value, color='r');
        ax[1].text(0.03,0.83,'{:.3f}'.format(period), transform=ax[1].transAxes, color='r') 
    
        transit_time = periodogram.transit_time[max_samp]  
        ts_folded = ts.fold(period=period, epoch_time=transit_time)  
        
        
        mean, median, stddev = sigma_clipped_stats(ts_folded['sap_flux'])  
        ts_folded['sap_flux_norm'] = ts_folded['sap_flux'] / median  
        
        ts_binned = aggregate_downsample(ts_folded, time_bin_size=0.03 * u.day) 
        
        ax[2].plot(ts_folded.time.jd, ts_folded['sap_flux_norm'], 'k.', markersize=1)
        ax[2].plot(ts_binned.time_bin_start.jd, ts_binned['sap_flux_norm'], 'r.', drawstyle='steps-post')
        ax[2].set_xlabel('Time (days)')
        ax[2].set_ylabel('Normalized flux')
#        plt.savefig('Spectra.png')
        
        freq[i] = (1/period)* 1000    #Fundamental frequency of each note
        amp[i] = dyn / 1000           #Amplitude of each note
        dur[i] = mean                 #Duration of each note
        i += 1
        
print ("KOI represented:",curves, "curves");

        

In [ ]:
period

In [ ]:
dyn

In [ ]:
duration

In [ ]:
freq

Duration=mean//////How to process amplitude in Music21?????

In [ ]:
dur

In [ ]:
amp

In [ ]:
dur_Mean = sigma_clipped_stats(dur)[0]
dur_Mean

In [ ]:
amp_Stddev = sigma_clipped_stats(amp)[1]
amp_Stddev

## Logarithmic transformation of the amplitudes

In [ ]:
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(np.log1p, validate=True)
ampLog = transformer.transform(amp)

In [ ]:
ampLog

## Normalization

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
amp_OK = np.round(min_max_scaler.fit_transform(ampLog)*127)

In [ ]:
amp_OK = amp_OK.astype(int)

In [ ]:
amp_OK

In [ ]:
for j in range (len(amp_OK)):
    if amp_OK[j] == 0:
        amp_OK[j] = 1

In [ ]:
amp_OK

In [ ]:
curves

## Score generation

In [ ]:
from music21 import *

In [ ]:
s = stream.Stream()
n = 120
s.append(tempo.MetronomeMark(number=n))

for k in range(curves):
    exec(f'n{k} = note.Note()')

In [ ]:
A4 = 440
C0 = A4*pow(2, -4.75)
name = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
def pitch(f):
    h = round(12*log2(f/C0))
    octave = h // 12
    n = h % 12
    return name[n]# + str(octave)

In [ ]:
pitch(440)

In [ ]:
durations = ["whole","half", "quarter", "eighth"]#, "16th"]#, "32nd", "64th"]
for i in range(curves):
    if dur[i] >= (dur_Mean * 2):
        figure = durations[0]
    if (dur[i] >= dur_Mean) and (dur[i] < dur_Mean * 2):
        figure = durations[1]
    if (dur[i] > dur_Mean / 2) and (dur[i] <= dur_Mean):
        figure = durations[2]
    if dur[i] <= (dur_Mean / 2):
        figure = durations[3]
   
    exec(f'n{i}.pitch.name = pitch(freq[i])')
    exec(f'n{i}.duration.type = figure')

In [ ]:
#velocities
for j in range(curves):
    amplitude = amp_OK[j][0]
    exec(f'n{j}.volume.velocity = amplitude')

In [ ]:
for j in range(curves):
    exec(f's.append(n{j})')
s.show()

In [ ]:
s.show('midi')

In [ ]:
s.show('musicxml')

In [ ]:
import session_info
session_info.show()